# Model Architecture


<img src="u-net-architecture.png">

Image taken from the original paper here: https://arxiv.org/abs/1505.04597

In [4]:
import torch
import torchvision.transforms.functional as F
import torch.nn as nn

In [7]:
FEATURES = [64, 128, 256, 512]

In [6]:
class PairConv(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(PairConv, self).__init__()
        self.conv = nn.Sequential(
                nn.Conv2d(in_channels, 
                          out_channels, 
                          kernel_size = 3,
                          stride = 1, 
                          padding = 1,
                          bias = False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace = True),
                nn.Conv2d(out_channels, 
                          out_channels, 
                          kernel_size = 3,
                          stride = 1, 
                          padding = 1,
                          bias = False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace = True)
            )
        
        def forward(self, x):
            return self.conv(x)

In [ ]:
class UNET(nn.Module):

    def __init__(self, in_channels = 3, out_channels = 1, features = FEATURES):
        super(UNET, self).__init__()
        self.contracting_path = nn.ModuleList()
        self.expansive_path = nn.ModuleList()
        self.max_pool = nn.MaxPool2d(kernel_size = 2, stride = 2,)

        for feature in features:
            self.contracting_path.append(PairConv(in_channels, feature))
            in_channels = feature

        for feature in reversed(features):
            self.expansive_path.append(
            nn.ConvTranspose2d(
                            feature*2,
                            feature,
                            kernel_size= 2,
                            stride = 2
                        )
                )
            self.expansive_path.append(PairConv(feature * 2, feature))
        
        self.bottom = PairConv(features[-1], features[-1] * 2)
        self.out_conv = nn.Conv2d(features[0], out_channels,kernel_size = 1)
